In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from collections import Counter
import json
import warnings
import re
warnings.filterwarnings('ignore')
import random
import statistics

from scipy.linalg import solve
import scipy
# import igraph as ig
from scipy.sparse import linalg as sla

In [2]:
def get_mat_file_info(filename):
    mat_file=loadmat(filename)
    # Get the number of files
    files=mat_file['dbs']['filename'].shape[1]
    file_names=[]
    for fi in range(files):
        file_names.append(mat_file['dbs']['filename'][0][fi][0])
    return file_names


# Choose file
## focusing on Head-tail for now 
filename='../data/mat_files/dbs_Head-Tail_20191013.mat'


# # Get filenames
files=get_mat_file_info(filename)

#dividing up the groups into 4
CCvsCC_female = []
FCvsCC_female = [] 
CCvsCC_male = [] 
FCvsCC_male = []
NacvsCC_male = []

for file in files:
#     print(file)
    if 'Female-CCvsCC' in file:
        CCvsCC_female.append(file)
    elif 'Female-FCvsCC' in file:
        FCvsCC_female.append(file)
    elif 'Male-CCvsCC' in file:
        CCvsCC_male.append(file) 
    elif 'Male-FCvsCC' in file:
        FCvsCC_male.append(file)  
    else:
        NacvsCC_male.append(file)
all_treatments = [CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male, NacvsCC_male]
print("Number of files for female-CCvsCC: " + str(len(CCvsCC_female)))
print("Number of files for female-FCvsCC: " + str(len(FCvsCC_female)))
print("Number of files for male-CCvsCC: " + str(len(CCvsCC_male)))
print("Number of files for male-FCvsCC: " + str(len(FCvsCC_male)))
print("Number of files for male-NACvsCC: " + str(len(NacvsCC_male)))


Number of files for female-CCvsCC: 10
Number of files for female-FCvsCC: 12
Number of files for male-CCvsCC: 12
Number of files for male-FCvsCC: 8
Number of files for male-NACvsCC: 8


In [3]:
#from https://rdrr.io/cran/igraph/src/R/structural.properties.R#sym-bonpow.dense
def bonacich_power_centrality(graph,exponent,tol,rescale):

    vg = 6 #number of mice

    #convert to matrix
    d = nx.to_scipy_sparse_matrix(graph)

    #adjacency matrix of size 6 (number of mice)
    id =np.identity(vg)
    
    degree_out = graph.out_degree()
    xarr = np.array(degree_out)
    degree_out_array = np.array([xarr[0][1],xarr[1][1],xarr[2][1],xarr[3][1],xarr[4][1],xarr[5][1]])
    degree_out_array = degree_out_array.astype(np.float)
    

    if round(np.linalg.det(id-(exponent*d))) == 0:
        return {}
    
    lu = sla.splu(id-(exponent*d))
    x = lu.solve(degree_out_array)
    
    if rescale:
        x = x/sum(x)
    else:
        x = x * (np.sqrt(vg/sum(x**2)))
        
    d =  {}
    for i in range(6):
        d[xarr[i][0]] = x[i]
    return d

In [4]:
def matlab_rand_net(in_deg,out_deg):

    n = len(in_deg)
    k = np.sum(in_deg).astype(np.int64)
    inInv = np.zeros((k,))
    outInv = np.zeros((k,))
    iIn = 0
    iOut = 0;

    for i in range(n):
        inInv[iIn:iIn+in_deg[i] ] = i+1
        outInv[iOut:iOut+out_deg[i] ] = i+1
        iIn = iIn+in_deg[i]
        iOut = iOut+out_deg[i]
    cij = np.eye(n).astype(np.int64)
    flag=-5
    edges = np.vstack([np.transpose(outInv[:k]), np.transpose(inInv[np.arange(k)])]).astype(np.int64)
    edges=edges-1
    for i in range(k):
        if cij[edges[0,i],edges[1,i]]==1:
            warningCounter = 1
            while 1:
                switchTo = np.random.randint(k)
                s=0
                if cij[edges[0,i],edges[1,switchTo]] ==0 and cij[edges[0,switchTo],edges[1,i]] ==0:
                    cij[edges[0,i],edges[1,switchTo]] = 1
                    if switchTo <= i:
                        cij[edges[0,switchTo],edges[1,switchTo]] = 0
                        cij[edges[0,switchTo],edges[1,i]] = 1
                    temp = edges[1,i]
                    edges[1,i] = edges[1,switchTo]
                    edges[1,switchTo] = temp;
                    break
                
                warningCounter = warningCounter+1;
                if warningCounter == 2*k^2:
                    flag = False
                    return cij,flag
        else:
            cij[edges[0,i],edges[1,i]] = 1
    cij = cij - np.eye(n)
    if flag==-5:
        flag=True
    return cij,flag

def nx_from_connectivity(A):
    G=nx.DiGraph()
    for i in range(np.shape(A)[0]):
        G.add_node(i)
    edges=np.where(A)
    for i in range(len(edges[0])):
        G.add_edge(edges[0][i],edges[1][i])
    return G

## Functions that calculate Z-Scores (Total or Divided in 2 groups)

In [11]:
def calculate_score(raw, network_measure_func, H,nrand, exponent, tol,rescale):
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
    
    #calculate network measure of 10000 random networks
    
    rand_values=np.zeros((nrand,))
    if network_measure_func == "NA":
        b = (bonacich_power_centrality(H,exponent,tol,rescale)).values()
    else:
        b = (network_measure_func(H)).values()
    
    raw_value=np.nanmean(list(b))
    
    if raw:
        return raw_value
    
    if network_measure_func == "NA" and not raw:
#         print("z-score all bonacich")
        results=np.zeros((6,1000))
        for i in range(nrand):
            done=False
            while not done:
                try:
                    #results[:,i] gets column i
                    results[:,i]=np.array(list(bonacich_power_centrality(nx.directed_configuration_model(din, dout, create_using=nx.DiGraph()), exponent, tol, rescale).values()))
                    done=True
                except:
                    False

        z_score = (np.mean((results[:,1]-np.mean(results[:,1:],1))/np.std(results[:,1:])))
        return z_score
    
    
    for i in range(nrand):
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        D = nx_from_connectivity(rnet)
        if network_measure_func == "NA":
            b = (bonacich_power_centrality(D,exponent,tol,rescale)).values()
        else:
            b = (network_measure_func(D)).items()
                
        rand_values[i] = np.nanmean(list(b))
    
    if np.round(statistics.stdev(rand_values)*1000) != 0:
        z_score = (raw_value- np.nanmean(rand_values)) / (np.std(rand_values)) 
    else:
        z_score = np.nan
    return z_score

def measures_individual(raw, network_measure_func, H, nrand, exponent, tol, rescale):
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
    if raw:
        return H_items

def measures_CCvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):

    #calculate actual network measures
    d_network = {'CC-1': [], 'CC-2': []}
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
    
    print(H_items)
    for key, value in H_items:   
        if key in ['CC1_1', 'CC1_2', 'CC1_3']:
            d_network['CC-1'].append(value)
        else:
            d_network['CC-2'].append(value)
    
    raw_value_CC1=np.nanmean(list(d_network['CC-1']))
    raw_value_CC2=np.nanmean(list(d_network['CC-2'])) 
  
    if raw:
        return [raw_value_CC1, raw_value_CC2]
    
    #calculate random network measures 
    rand_values_CC1=np.zeros((nrand,))
    rand_values_CC2=np.zeros((nrand,))

    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC-1': [], 'CC-2': []}
    
        
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()
        
        #calculate network measure
        for key, value in d_items:
            if key in ['CC1_1', 'CC1_2', 'CC1_3']:
                d_random['CC-1'].append(value)
            else:
                d_random['CC-2'].append(value)
            
        rand_values_CC1[i] = np.nanmean(list(d_random['CC-1']))
        rand_values_CC2[i] = np.nanmean(list(d_random['CC-1']))
        
    
    if np.round(statistics.stdev(rand_values_CC1)*1000) != 0:
        z_score_CC1 = (raw_value_CC1 -np.nanmean(rand_values_CC1)) / (np.std(rand_values_CC1)) 
    else:
        z_score_CC1 = np.nan

    
    if np.round(statistics.stdev(rand_values_CC2)*1000) != 0:
        z_score_CC2 = (raw_value_CC2- np.nanmean(rand_values_CC2)) / (np.std(rand_values_CC2)) 
    else:
        z_score_CC2 = np.nan
        
    return [z_score_CC1, z_score_CC2]
        
        
def measures_NACvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):
    
    #calculate actual network measures
    d_network = {'CC': [], 'NAC': []}
    
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
#     print(H_items)
    for key, value in H_items:   
#         print(key)
        if "CC" in str(key):
            d_network['CC'].append(value)
        elif "Nac" in str(key):
            d_network['NAC'].append(value)
    
    raw_value_NAC=np.nanmean(list(d_network['NAC']))
    raw_value_CC=np.nanmean(list(d_network['CC'])) 

    if raw:
        return [raw_value_CC, raw_value_NAC]
    
    #calculate random network measures 
    rand_values_CC=np.zeros((nrand,))
    rand_values_NAC=np.zeros((nrand,))
    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC': [], 'NAC': []}
        

        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()  
       
        #calculate network measure
        for key, value in d_items:
            if "CC" in key:
                d_random['CC'].append(value)
            elif "Nac" in key:
                d_random['NAC'].append(value)
            
        rand_values_CC[i] = np.nanmean(list(d_random['CC']))
        rand_values_NAC[i] = np.nanmean(list(d_random['NAC']))

    if np.round(statistics.stdev(rand_values_CC)*1000) != 0:
        z_score_CC = (raw_value_CC -np.nanmean(rand_values_CC)) / (np.std(rand_values_CC)) 
    else:
        z_score_CC = np.nan

    if np.round(statistics.stdev(rand_values_NAC)*1000) != 0:
        z_score_NAC = (raw_value_NAC- np.nanmean(rand_values_NAC)) / (np.std(rand_values_NAC)) 
    else:
        z_score_NAC = np.nan
        
    return [z_score_CC, z_score_NAC]


def measures_FCvsCC(raw, network_measure_func, H, nrand, exponent, tol,rescale):
    
    #calculate actual network measures
    d_network = {'CC': [], 'FC': []}
    if network_measure_func == "NA":
        H_items = (bonacich_power_centrality(H,exponent,tol,rescale)).items()
    else:
        H_items = (network_measure_func(H)).items()
    
    for key, value in H_items:   
        if "FC" in key:
            d_network['FC'].append(value)
        elif "CC" in key:
            d_network['CC'].append(value)
    
    raw_value_FC=np.nanmean(list(d_network['FC']))
    raw_value_CC=np.nanmean(list(d_network['CC'])) 

    if raw: 
        return [raw_value_CC, raw_value_FC]
    #calculate random network measures 
    rand_values_CC=np.zeros((nrand,))
    rand_values_FC=np.zeros((nrand,))
    
    din=list(d for n, d in H.in_degree())
    dout = list(d for n, d in H.out_degree())
  
    for i in range(nrand):
        #reset random dicts every iteration 
        d_random = {'CC': [], 'FC': []}
        
        flag=False
        while flag==False:
            rnet,flag=matlab_rand_net(din,dout)
        
        random_nodes = list(random.sample(H.nodes,len(H.nodes)))
        map_nodes = {} 
        for count, node in enumerate(random_nodes):
            map_nodes[count] = node
        
        D = nx_from_connectivity(rnet)
        D =nx.relabel_nodes(D,map_nodes)
        
        if network_measure_func == "NA":
            d_items = (bonacich_power_centrality(D,exponent,tol,rescale)).items()
        else:
            d_items = (network_measure_func(D)).items()
        
        #calculate network measure
        for key, value in d_items:
            if "CC" in key:
                d_random['CC'].append(value)
            elif "FC" in key:
                d_random['FC'].append(value)
            
        rand_values_CC[i] = np.nanmean(list(d_random['CC']))
        rand_values_FC[i] = np.nanmean(list(d_random['FC']))
        
    
    if np.round(statistics.stdev(rand_values_CC)*1000) != 0:
        z_score_CC = (raw_value_CC -np.nanmean(rand_values_CC)) / (np.std(rand_values_CC)) 
    else:
        z_score_CC = np.nan

    
    if np.round(statistics.stdev(rand_values_FC)*1000) != 0:
        z_score_FC = (raw_value_FC- np.nanmean(rand_values_FC)) / (np.std(rand_values_FC)) 
    else:
        z_score_FC = np.nan
        
    return [z_score_CC, z_score_FC]



In [39]:
def load_iterations_from_mat_file(filename,file):
    mat_file=loadmat(filename)
    # Get the number of files
    files=mat_file['dbs']['filename'].shape[1]
    file_names=[]
    for fi in range(files):
        file_names.append(mat_file['dbs']['filename'][0][fi][0])
    file_num=np.where(np.array(file_names)==file)[0][0]
        
    if len(mat_file['dbs']['networks'][0][file_num].shape)<3:
        mat_file['dbs']['networks'][0][file_num]=mat_file['dbs']['networks'][0][file_num][:,:,np.newaxis]
        
    G_list=[]
    # Get the number of mice
    nmice=mat_file['dbs']['networks'][0][file_num][:,:,0].shape[0]
    # Get the ids
    ids_t=mat_file['dbs']['ids'][0][file_num][0]
    ids=[]
    for id in range(nmice):
        ids.append(ids_t[id][0])
    types=np.unique(ids)
    type_count=np.ones((len(types),),np.int64)
    mapping = {}
    for i in range(nmice):
        mapping[i]=ids[i]+'_'+str(type_count[np.where(types==ids[i])[0]][0])
        type_count[np.where(types==ids[i])[0]]+=1
    # Get the number of iterations
    iters=mat_file['dbs']['networks'][0][file_num].shape[2]
    
    
    # Make the graph
    for iter in range(iters):
        mat_net=mat_file['dbs']['networks'][0][file_num][:,:,iter]
        mat_net[mat_net>0]=1
        G=nx.from_numpy_matrix(mat_net,create_using=nx.DiGraph())
        G=nx.relabel_nodes(G,mapping)
        G_list.append(G)
    return G_list



def get_values(treatment_file, interaction_file):
    lst = [] 
    if 'Female-CCvsCC' in treatment_file:
        lst = ['Female', 'CCvsCC']
    elif 'Female-FCvsCC' in treatment_file:
        lst = ['Female', 'FCvsCC']
    elif 'Male-CCvsCC' in treatment_file:
        lst = ['Male', 'CCvsCC']
    elif 'Male-FCvsCC' in treatment_file:
        lst = ['Male', 'FCvsCC']
    elif 'NacvsCC' in treatment_file:
        lst = ['Male', 'NACvsCC']
    if 'Head-Tail' in interaction_file:
        return lst + ['Head-Tail']
    if 'Head-Head' in interaction_file:
        return lst + ['Head-Head']
    if 'Centers-Close' in interaction_file:
        return lst + ['Centers-Close']
    if 'Oriented-Towards' in interaction_file:
        return lst + ['Oriented-Towards']

def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, random_iterations):
    date = "20191310"
    print("Properties of current analysis:")
    print("Raw: ", str(raw))
    print("Split groups calculation: ", str(not All))
    print("Network Measure: ", str(network_measures[0][0]))
    print("----------------------")
    df_all = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
    df_split = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File', 'Iteration', 'Total_Iterations'])
    if raw:
        type = "raw_"
    else:
        type = "_z-score_" + str(random_iterations) + "_"
    if network_measures[0][1] == "NA":
        type += "exponent_" + str(exponent)
        
    for treatment in treatments: 
    
        # Get filenames
        files=get_mat_file_info(treatment)
    
        #dividing up the groups into 4
        CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male, NACvsCC_male  = ([] for i in range(5))
        for file in files:
            if 'Female-CCvsCC' in file:
                CCvsCC_female.append(file)
            elif 'Female-FCvsCC' in file:
                FCvsCC_female.append(file)
            elif 'Male-CCvsCC' in file:
                CCvsCC_male.append(file) 
            elif 'Male-FCvsCC' in file:
                FCvsCC_male.append(file)
            elif 'Male-NacvsCC' in file:
                NACvsCC_male.append(file)
        all_treatments = [NACvsCC_male, CCvsCC_female, FCvsCC_female, CCvsCC_male, FCvsCC_male]
        total_iterations = 0
        t = 1
        for x in range(len(all_treatments)):
            mean_trial_raw = {}
            mean_trial_zscore = {} 
            for i in range(len(all_treatments[x])):
                file=all_treatments[x][i]
                print(file)
                print(treatment)
                
                stability_all = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                stability_split = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                
                # Get the network measures
                G_list=load_iterations_from_mat_file(treatment,file)
                
                
                df_all_temp = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                df_split_temp_1 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                df_split_temp_2 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Care', 'Group', 'File', 'Iteration', 'Total_Iterations'])

                
                for ii in range(len(G_list)):
                    
                    #G_list is total number of iterations
                    
                    
                    gender, strain, interaction = get_values(all_treatments[x][i], treatment)

                    total_iterations += 1

                    G=G_list[ii]
                    if len(G_list) == 26 and ii == 7:
                        continue
                     
                    
                    if All:
                        #calculate z-score for all network, and add to df_zscore
                        for i in range(len(network_measures)):
                            z_score = calculate_score(raw, network_measures[i][1],G,random_iterations,exponent,tol,rescale) 
                            df_all_temp.loc[len(df_all_temp)]=[network_measures[i][0], z_score, interaction, gender, strain, t, file, ii+1, len(G_list)]
                            stability_all.loc[len(stability_all)] = [network_measures[i][0], z_score, interaction, gender, strain, t, file, ii+1, len(G_list)]
                    else:                         
                            
                        #split z-score into FC and CC or CC1 and CC2, add to df_zscore  
                        individual = measures_individual(raw, network_measures[0][1], G, random_iterations, exponent,tol,rescale)
                        for name, score in individual:
                            df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score, interaction, gender, strain, name, t, file, ii+1, len(G_list)]
#                             df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC, interaction, gender, strain, "CC", t, file, ii+1, len(G_list)]
#                             df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_NAC, interaction, gender, strain, "NAC", t, file, ii+1, len(G_list)]
                        
#                         elif strain == "CCvsCC":
#                             score_CC1, score_CC2 = measures_CCvsCC(raw, network_measures[0][1], G, random_iterations, exponent,tol,rescale)
#                             df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC1, interaction, gender, strain, "CC1", t, file, ii+1, len(G_list)]
#                             df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_CC2, interaction, gender, strain, "CC2", t, file, ii+1, len(G_list)]
                        
#                         elif strain == "FCvsCC":
#                             score_CC, score_FC = measures_FCvsCC(raw, network_measures[0][1], G, random_iterations ,exponent,tol,rescale)
#                             df_split_temp_1.loc[len(df_split_temp_1)]=[network_measures[0][0], score_CC, interaction, gender, strain, "CC", t, file, ii+1, len(G_list)]
#                             df_split_temp_2.loc[len(df_split_temp_2)]=[network_measures[0][0], score_FC, interaction, gender, strain, "FC", t, file, ii+1, len(G_list)]
                t+= 1
                print(df_split_temp_1)
                if All:
                    name = "_" + str(type) + str(interaction) + "_" + str(gender) + "_" + str(strain) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + ".csv"
                    print("------------------------------- name " + str(name))
                    df_all_temp.to_csv("../data/consistency/" + str(str(network_measures[0][0]).replace(" ", "_")) + "/consistency_ALL" + str(name))

#                     new_list = (df_all_temp.iloc[0]).tolist()
#                     if df_all_temp['Score'].mean() == 0:
#                         print(df_all_temp['Score'])
#                     new_list[1] = df_all_temp["Score"].mean()
#                     print(new_list)

#                     df_all_zscore.loc[len(df_all_zscore)]=new_list
                    df_all_temp = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
#                     df_all_zscore.to_csv("../data/temp_ALL_score.csv")
                    
                else: 
#                     frames = [df_split_temp_1, df_split_temp_2]
#                     result = pd.concat(frames)
                    name = "_individual_" + str(type) + str(interaction) + "_" + str(gender) + "_" + str(strain) + "_" + str(str(network_measures[0][0]).replace(" ", "_")) + ".csv"
                    print("------------------------------- name " + str(name))

                    df_split_temp_1.to_csv("../data/consistency/" + str(str(network_measures[0][0]).replace(" ", "_")) + "/consistency_SPLIT" + str(name))
                                               
                    df_split_temp_1 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                                        
                    df_split_temp_2 = pd.DataFrame([], columns = ['Network Measure', 'Score', 'Interaction', 'Gender', 'Strain', 'Group', 'File', 'Iteration', 'Total_Iterations'])
                        


#     if network_measures[0][1] == "NA":
#         type += "exponent_" + str(exponent)
#     if All:
#         df_all_zscore.to_csv("/raid/saraelshawa/October_Work/data/" + str(date) + "_" + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_ALL.csv")
#         print("data saved in file: " + str("./data/" + str(date)  + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_ALL.csv"))
#         return df_all_zscore
    
#     df_split_zscore.to_csv("/raid/saraelshawa/October_Work/data/" + str(date) + "_"  + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_SPLIT.csv")
#     print("data saved in file: " + str("./data/" + str(date) + str(type) + "_" + str(interaction) + "_" + str(str(network_measures[0][0]).replace(" ", "")) + "_SPLIT.csv"))
#     return df_split_zscore

In [19]:
# %%time
# # def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# # treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
# treatments = [['../data/mat_files/dbs_Head-Tail_20191013.mat']]
# for x in treatments:
#     print(x)
#     df_zscore_ev_split = all_treatment_analysis(x, False, [['eigenvector centrality', nx.eigenvector_centrality_numpy]], False, 0, 1e-07,True, True, 0)

['../data/mat_files/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  eigenvector centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
CC_1
-3.841724429933899e-17
CC_2
0.5211212421739552
CC_3
0.5211212421739552
Nac_1
0.3555755127006844
Nac_2
0.24261905867607989
Nac_3
0.5211212421739554


TypeError: list indices must be integers or slices, not str

In [38]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [['../data/mat_files/dbs_Head-Tail_20191013.mat']]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['betweenness centrality', nx.betweenness_centrality]], False, 0, 1e-07,True, True, 0)

['../data/mat_files/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  betweenness centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
            Network Measure  Score Interaction Gender   Strain   Care Group  \
0    betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_1     1   
1    betweenness centrality  0.150   Head-Tail   Male  NACvsCC   CC_2     1   
2    betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_3     1   
3    betweenness centrality  0.100   Head-Tail   Male  NACvsCC  Nac_1     1   
4    betweenness centrality  0.150   Head-Tail   Male  NACvsCC  Nac_2     1   
5    betweenness centrality  0.050   Head-Tail   Male  NACvsCC  Nac_3     1   
6    betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_1     1   
7    betweenness centrality  0

            Network Measure  Score Interaction Gender   Strain   Care Group  \
0    betweenness centrality  0.250   Head-Tail   Male  NACvsCC   CC_1     2   
1    betweenness centrality  0.300   Head-Tail   Male  NACvsCC   CC_2     2   
2    betweenness centrality  0.150   Head-Tail   Male  NACvsCC   CC_3     2   
3    betweenness centrality  0.050   Head-Tail   Male  NACvsCC  Nac_1     2   
4    betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_2     2   
5    betweenness centrality  0.150   Head-Tail   Male  NACvsCC  Nac_3     2   
6    betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_1     2   
7    betweenness centrality  0.300   Head-Tail   Male  NACvsCC   CC_2     2   
8    betweenness centrality  0.250   Head-Tail   Male  NACvsCC   CC_3     2   
9    betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_1     2   
10   betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_2     2   
11   betweenness centrality  0.250   Head-Tail   Mal

           Network Measure  Score Interaction Gender   Strain   Care Group  \
0   betweenness centrality   0.00   Head-Tail   Male  NACvsCC   CC_1     3   
1   betweenness centrality   0.40   Head-Tail   Male  NACvsCC   CC_2     3   
2   betweenness centrality   0.00   Head-Tail   Male  NACvsCC   CC_3     3   
3   betweenness centrality   0.15   Head-Tail   Male  NACvsCC  Nac_1     3   
4   betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_2     3   
5   betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_3     3   
6   betweenness centrality   0.00   Head-Tail   Male  NACvsCC   CC_1     3   
7   betweenness centrality   0.45   Head-Tail   Male  NACvsCC   CC_2     3   
8   betweenness centrality   0.00   Head-Tail   Male  NACvsCC   CC_3     3   
9   betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_1     3   
10  betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_2     3   
11  betweenness centrality   0.00   Head-Tail   Male  NACvsCC  N

           Network Measure  Score Interaction Gender   Strain   Care Group  \
0   betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_1     4   
1   betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_2     4   
2   betweenness centrality  0.250   Head-Tail   Male  NACvsCC   CC_3     4   
3   betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_1     4   
4   betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_2     4   
5   betweenness centrality  0.150   Head-Tail   Male  NACvsCC  Nac_3     4   
6   betweenness centrality  0.000   Head-Tail   Male  NACvsCC   CC_1     4   
7   betweenness centrality  0.350   Head-Tail   Male  NACvsCC   CC_2     4   
8   betweenness centrality  0.300   Head-Tail   Male  NACvsCC   CC_3     4   
9   betweenness centrality  0.250   Head-Tail   Male  NACvsCC  Nac_1     4   
10  betweenness centrality  0.000   Head-Tail   Male  NACvsCC  Nac_2     4   
11  betweenness centrality  0.150   Head-Tail   Male  NACvsCC  N

           Network Measure  Score Interaction Gender   Strain   Care Group  \
0   betweenness centrality   0.35   Head-Tail   Male  NACvsCC   CC_1     5   
1   betweenness centrality   0.15   Head-Tail   Male  NACvsCC   CC_2     5   
2   betweenness centrality   0.15   Head-Tail   Male  NACvsCC   CC_3     5   
3   betweenness centrality   0.35   Head-Tail   Male  NACvsCC  Nac_1     5   
4   betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_2     5   
5   betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_3     5   
6   betweenness centrality   0.35   Head-Tail   Male  NACvsCC   CC_1     5   
7   betweenness centrality   0.15   Head-Tail   Male  NACvsCC   CC_2     5   
8   betweenness centrality   0.15   Head-Tail   Male  NACvsCC   CC_3     5   
9   betweenness centrality   0.35   Head-Tail   Male  NACvsCC  Nac_1     5   
10  betweenness centrality   0.00   Head-Tail   Male  NACvsCC  Nac_2     5   
11  betweenness centrality   0.00   Head-Tail   Male  NACvsCC  N

...

           Network Measure  Score Interaction Gender  Strain  Care Group  \
0   betweenness centrality  0.350   Head-Tail   Male  FCvsCC  CC_1    46   
1   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  CC_2    46   
2   betweenness centrality  0.500   Head-Tail   Male  FCvsCC  CC_3    46   
3   betweenness centrality  0.500   Head-Tail   Male  FCvsCC  FC_1    46   
4   betweenness centrality  0.500   Head-Tail   Male  FCvsCC  FC_2    46   
5   betweenness centrality  0.350   Head-Tail   Male  FCvsCC  FC_3    46   
6   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  CC_1    46   
7   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  CC_2    46   
8   betweenness centrality  0.250   Head-Tail   Male  FCvsCC  CC_3    46   
9   betweenness centrality  0.450   Head-Tail   Male  FCvsCC  FC_1    46   
10  betweenness centrality  0.350   Head-Tail   Male  FCvsCC  FC_2    46   
11  betweenness centrality  0.000   Head-Tail   Male  FCvsCC  FC_3    46   
12  betweenn

           Network Measure  Score Interaction Gender  Strain  Care Group  \
0   betweenness centrality  0.175   Head-Tail   Male  FCvsCC  CC_1    47   
1   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  CC_2    47   
2   betweenness centrality  0.075   Head-Tail   Male  FCvsCC  CC_3    47   
3   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  FC_1    47   
4   betweenness centrality  0.400   Head-Tail   Male  FCvsCC  FC_2    47   
5   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  FC_3    47   
6   betweenness centrality  0.175   Head-Tail   Male  FCvsCC  CC_1    47   
7   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  CC_2    47   
8   betweenness centrality  0.075   Head-Tail   Male  FCvsCC  CC_3    47   
9   betweenness centrality  0.000   Head-Tail   Male  FCvsCC  FC_1    47   
10  betweenness centrality  0.400   Head-Tail   Male  FCvsCC  FC_2    47   
11  betweenness centrality  0.000   Head-Tail   Male  FCvsCC  FC_3    47   
12  betweenn

           Network Measure     Score Interaction Gender  Strain  Care Group  \
0   betweenness centrality  0.000000   Head-Tail   Male  FCvsCC  CC_1    48   
1   betweenness centrality  0.066667   Head-Tail   Male  FCvsCC  CC_2    48   
2   betweenness centrality  0.016667   Head-Tail   Male  FCvsCC  CC_3    48   
3   betweenness centrality  0.000000   Head-Tail   Male  FCvsCC  FC_1    48   
4   betweenness centrality  0.016667   Head-Tail   Male  FCvsCC  FC_2    48   
5   betweenness centrality  0.000000   Head-Tail   Male  FCvsCC  FC_3    48   
6   betweenness centrality  0.000000   Head-Tail   Male  FCvsCC  CC_1    48   
7   betweenness centrality  0.175000   Head-Tail   Male  FCvsCC  CC_2    48   
8   betweenness centrality  0.075000   Head-Tail   Male  FCvsCC  CC_3    48   
9   betweenness centrality  0.000000   Head-Tail   Male  FCvsCC  FC_1    48   
10  betweenness centrality  0.150000   Head-Tail   Male  FCvsCC  FC_2    48   
11  betweenness centrality  0.250000   Head-Tail   M

            Network Measure  Score Interaction Gender  Strain  Care Group  \
0    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_1    49   
1    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_2    49   
2    betweenness centrality   0.25   Head-Tail   Male  FCvsCC  CC_3    49   
3    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_1    49   
4    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_2    49   
5    betweenness centrality   0.25   Head-Tail   Male  FCvsCC  FC_3    49   
6    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_1    49   
7    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_2    49   
8    betweenness centrality   0.55   Head-Tail   Male  FCvsCC  CC_3    49   
9    betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_1    49   
10   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_2    49   
11   betweenness centrality   0.30   Head-Tail   Male  FCvsCC  FC_3    49   

           Network Measure  Score Interaction Gender  Strain  Care Group  \
0   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_1    50   
1   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_2    50   
2   betweenness centrality   0.15   Head-Tail   Male  FCvsCC  CC_3    50   
3   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_1    50   
4   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_2    50   
5   betweenness centrality   0.25   Head-Tail   Male  FCvsCC  FC_3    50   
6   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_1    50   
7   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  CC_2    50   
8   betweenness centrality   0.25   Head-Tail   Male  FCvsCC  CC_3    50   
9   betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_1    50   
10  betweenness centrality   0.00   Head-Tail   Male  FCvsCC  FC_2    50   
11  betweenness centrality   0.25   Head-Tail   Male  FCvsCC  FC_3    50   
12  betweenn

In [41]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [['../data/mat_files/dbs_Head-Tail_20191013.mat']]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['clustering centrality', nx.clustering]], False, 0, 1e-07,True, True, 0)

['../data/mat_files/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  clustering centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat


NetworkXNotImplemented: not implemented for directed type

In [42]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [['../data/mat_files/dbs_Head-Tail_20191013.mat']]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['outdegree centrality', nx.out_degree_centrality]], False, 0, 1e-07,True, True, 0)

['../data/mat_files/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  outdegree centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
          Network Measure  Score Interaction Gender   Strain   Care Group  \
0    outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     1   
1    outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     1   
2    outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_3     1   
3    outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     1   
4    outdegree centrality    0.6   Head-Tail   Male  NACvsCC  Nac_2     1   
5    outdegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     1   
6    outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     1   
7    outdegree centrality    0.4   Head-Tail   M

          Network Measure  Score Interaction Gender   Strain   Care Group  \
0    outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_1     2   
1    outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     2   
2    outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     2   
3    outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     2   
4    outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     2   
5    outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     2   
6    outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     2   
7    outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_2     2   
8    outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     2   
9    outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     2   
10   outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     2   
11   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     2   

         Network Measure  Score Interaction Gender   Strain   Care Group  \
0   outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     3   
1   outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_2     3   
2   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     3   
3   outdegree centrality    0.6   Head-Tail   Male  NACvsCC  Nac_1     3   
4   outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     3   
5   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     3   
6   outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     3   
7   outdegree centrality    0.8   Head-Tail   Male  NACvsCC   CC_2     3   
8   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     3   
9   outdegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_1     3   
10  outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     3   
11  outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     3   
12  outdegre

         Network Measure  Score Interaction Gender   Strain   Care Group  \
0   outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_1     4   
1   outdegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_2     4   
2   outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     4   
3   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     4   
4   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     4   
5   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     4   
6   outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_1     4   
7   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_2     4   
8   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     4   
9   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     4   
10  outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     4   
11  outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     4   
12  outdegre

         Network Measure  Score Interaction Gender   Strain   Care Group  \
0   outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_1     5   
1   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_2     5   
2   outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     5   
3   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     5   
4   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     5   
5   outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_3     5   
6   outdegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_1     5   
7   outdegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_2     5   
8   outdegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     5   
9   outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     5   
10  outdegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     5   
11  outdegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_3     5   
12  outdegre

...

         Network Measure  Score Interaction Gender  Strain  Care Group  \
0   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_1    46   
1   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    46   
2   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    46   
3   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    46   
4   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    46   
5   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    46   
6   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_1    46   
7   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    46   
8   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    46   
9   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    46   
10  outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    46   
11  outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    46   
12  outdegree centrality    0.4   Head

         Network Measure  Score Interaction Gender  Strain  Care Group  \
0   outdegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_1    47   
1   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    47   
2   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    47   
3   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    47   
4   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_2    47   
5   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_3    47   
6   outdegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_1    47   
7   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    47   
8   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    47   
9   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    47   
10  outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_2    47   
11  outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_3    47   
12  outdegree centrality    0.6   Head

         Network Measure  Score Interaction Gender  Strain  Care Group  \
0   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    48   
1   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_2    48   
2   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    48   
3   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    48   
4   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    48   
5   outdegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_3    48   
6   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    48   
7   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_2    48   
8   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    48   
9   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    48   
10  outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_2    48   
11  outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    48   
12  outdegree centrality    0.0   Head

          Network Measure  Score Interaction Gender  Strain  Care Group  \
0    outdegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    49   
1    outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    49   
2    outdegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_3    49   
3    outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    49   
4    outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    49   
5    outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    49   
6    outdegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    49   
7    outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    49   
8    outdegree centrality    0.8   Head-Tail   Male  FCvsCC  CC_3    49   
9    outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    49   
10   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    49   
11   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    49   
12   outdegree centrality

         Network Measure  Score Interaction Gender  Strain  Care Group  \
0   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    50   
1   outdegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_2    50   
2   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    50   
3   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    50   
4   outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_2    50   
5   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    50   
6   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    50   
7   outdegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_2    50   
8   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    50   
9   outdegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    50   
10  outdegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_2    50   
11  outdegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_3    50   
12  outdegree centrality    0.2   Head

In [43]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [['../data/mat_files/dbs_Head-Tail_20191013.mat']]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, False, [['indegree centrality', nx.in_degree_centrality]], False, 0, 1e-07,True, True, 0)

['../data/mat_files/dbs_Head-Tail_20191013.mat']
Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  indegree centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
         Network Measure  Score Interaction Gender   Strain   Care Group  \
0    indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     1   
1    indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     1   
2    indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_3     1   
3    indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     1   
4    indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     1   
5    indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     1   
6    indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     1   
7    indegree centrality    0.4   Head-Tail   Male  NACvs

         Network Measure  Score Interaction Gender   Strain   Care Group  \
0    indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     2   
1    indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     2   
2    indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     2   
3    indegree centrality    0.6   Head-Tail   Male  NACvsCC  Nac_1     2   
4    indegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     2   
5    indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     2   
6    indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     2   
7    indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     2   
8    indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     2   
9    indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_1     2   
10   indegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     2   
11   indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     2   
12   indegre

        Network Measure  Score Interaction Gender   Strain   Care Group  \
0   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     3   
1   indegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_2     3   
2   indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_3     3   
3   indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     3   
4   indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     3   
5   indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     3   
6   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     3   
7   indegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_2     3   
8   indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_3     3   
9   indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     3   
10  indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_2     3   
11  indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     3   
12  indegree centrality  

        Network Measure  Score Interaction Gender   Strain   Care Group  \
0   indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     4   
1   indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     4   
2   indegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_3     4   
3   indegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_1     4   
4   indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_2     4   
5   indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     4   
6   indegree centrality    0.0   Head-Tail   Male  NACvsCC   CC_1     4   
7   indegree centrality    0.4   Head-Tail   Male  NACvsCC   CC_2     4   
8   indegree centrality    0.6   Head-Tail   Male  NACvsCC   CC_3     4   
9   indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_1     4   
10  indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_2     4   
11  indegree centrality    0.2   Head-Tail   Male  NACvsCC  Nac_3     4   
12  indegree centrality  

        Network Measure  Score Interaction Gender   Strain   Care Group  \
0   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     5   
1   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_2     5   
2   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     5   
3   indegree centrality    0.6   Head-Tail   Male  NACvsCC  Nac_1     5   
4   indegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     5   
5   indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     5   
6   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_1     5   
7   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_2     5   
8   indegree centrality    0.2   Head-Tail   Male  NACvsCC   CC_3     5   
9   indegree centrality    0.6   Head-Tail   Male  NACvsCC  Nac_1     5   
10  indegree centrality    0.0   Head-Tail   Male  NACvsCC  Nac_2     5   
11  indegree centrality    0.4   Head-Tail   Male  NACvsCC  Nac_3     5   
12  indegree centrality  

...

        Network Measure  Score Interaction Gender  Strain  Care Group  \
0   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    46   
1   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    46   
2   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    46   
3   indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_1    46   
4   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    46   
5   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    46   
6   indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    46   
7   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    46   
8   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    46   
9   indegree centrality    0.8   Head-Tail   Male  FCvsCC  FC_1    46   
10  indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    46   
11  indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    46   
12  indegree centrality    0.0   Head-Tail   Male  

        Network Measure  Score Interaction Gender  Strain  Care Group  \
0   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    47   
1   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    47   
2   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    47   
3   indegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    47   
4   indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_2    47   
5   indegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    47   
6   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    47   
7   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    47   
8   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    47   
9   indegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    47   
10  indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_2    47   
11  indegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    47   
12  indegree centrality    0.2   Head-Tail   Male  

        Network Measure  Score Interaction Gender  Strain  Care Group  \
0   indegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_1    48   
1   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    48   
2   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    48   
3   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    48   
4   indegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_2    48   
5   indegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_3    48   
6   indegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_1    48   
7   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    48   
8   indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_3    48   
9   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    48   
10  indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    48   
11  indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    48   
12  indegree centrality    0.6   Head-Tail   Male  

         Network Measure  Score Interaction Gender  Strain  Care Group  \
0    indegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_1    49   
1    indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_2    49   
2    indegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    49   
3    indegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    49   
4    indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    49   
5    indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_3    49   
6    indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_1    49   
7    indegree centrality    0.2   Head-Tail   Male  FCvsCC  CC_2    49   
8    indegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    49   
9    indegree centrality    0.0   Head-Tail   Male  FCvsCC  FC_1    49   
10   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_2    49   
11   indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_3    49   
12   indegree centrality    0.2   Head

        Network Measure  Score Interaction Gender  Strain  Care Group  \
0   indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    50   
1   indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_2    50   
2   indegree centrality    0.4   Head-Tail   Male  FCvsCC  CC_3    50   
3   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    50   
4   indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_2    50   
5   indegree centrality    0.4   Head-Tail   Male  FCvsCC  FC_3    50   
6   indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_1    50   
7   indegree centrality    0.0   Head-Tail   Male  FCvsCC  CC_2    50   
8   indegree centrality    0.6   Head-Tail   Male  FCvsCC  CC_3    50   
9   indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_1    50   
10  indegree centrality    0.6   Head-Tail   Male  FCvsCC  FC_2    50   
11  indegree centrality    0.2   Head-Tail   Male  FCvsCC  FC_3    50   
12  indegree centrality    0.0   Head-Tail   Male  

In [53]:
!pwd

/home/sara/repos/Mouse-SNA/code


In [55]:
%%time

import time
time.ctime()
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/mat_files/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["../data/mat_files/dbs_Head-Tail_20191013.mat"]]
for t in treatments:
    x = [0, 1, -1, 0.1, -0.1]
    for exponent in x:
        df_zscore_ev_split = all_treatment_analysis(t, False, [['power centrality', "NA"]], False, exponent, 1e-07,True, True, 0)

Properties of current analysis:
Raw:  True
Split groups calculation:  True
Network Measure:  power centrality
----------------------
fixed_Male-NacvsCC_3vs3Males_P60_CCvsNAC_Group8V_Interaction_30min_cam_2_Sat_09_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group1F_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
../data/mat_files/dbs_He

------------------------------- name _raw_exponent_0Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group1_Interaction_30min_cam_2_Tue_01_08_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group2_Interaction_30min_cam_2_Fri_01_11_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group3_Interaction_30min_cam_2_Tue_01_15_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group4J_Interaction_30min_cam_2_Tue_03_05_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
-------------------------------

------------------------------- name _raw_exponent_1Head-Tail_Female_CCvsCC_power_centrality.csv
fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group8M_interaction_30min_cam_2_Thu_08_01_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_1Head-Tail_Female_CCvsCC_power_centrality.csv
fixed_Female-CCvsCC_3vs3Females_P60_CCvsCC_Group9L_interaction_30min_cam_2_Thu_08_01_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_1Head-Tail_Female_CCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group10J_interaction_30min_cam_2_Tue_07_16_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_1Head-Tail_Female_FCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group11K_interaction_30min_cam_2_Tue_07_16_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
--

------------------------------- name _raw_exponent_-1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group5O_interaction_30min_cam_2_Tue_08_06_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NACvsCC_Group6P_interaction_30min_cam_2_Wed_08_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group1F_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group2G_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------

------------------------------- name _raw_exponent_-1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group3_Interaction_30min_cam_2_Tue_01_15_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group4J_Interaction_30min_cam_2_Tue_03_05_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group5M_Interaction_30min_cam_2_Wed_03_27_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group6N_Interaction_30min_cam_2_Tue_04_16_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
-------------------------

------------------------------- name _raw_exponent_0.1Head-Tail_Female_FCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group12H_interaction_30min_cam_2_Wed_07_17_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0.1Head-Tail_Female_FCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group1_Interaction_0_30_min_cam_2_Wed_09_26_2018_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0.1Head-Tail_Female_FCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group2_0-30mins_interaction_cam_2_Wed_10_10_2018_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_0.1Head-Tail_Female_FCvsCC_power_centrality.csv
fixed_Female-FCvsCC_3vs3Females_P60_FCvsCC_Group3_Interaction_30min_cam_2_Thu_11_01_2018_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191

------------------------------- name _raw_exponent_-0.1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group2G_interaction_30min_cam_2_Wed_07_24_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group3I_interaction_30min_cam_2_Tue_07_30_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group7R_interaction_30min_cam_2_Wed_08_28_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_NACvsCC_power_centrality.csv
fixed_Male-NacvsCC_3vs3Males_P60_NacvsCC_Group9X_interaction_30min_cam_2_Wed_09_18_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
----

------------------------------- name _raw_exponent_-0.1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group6N_Interaction_30min_cam_2_Tue_04_16_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group7Q_Interaction_30min_cam_2_Tue_04_16_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group8T_Interaction_30min_cam_2_Tue_04_23_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
------------------------------- name _raw_exponent_-0.1Head-Tail_Male_CCvsCC_power_centrality.csv
fixed_Male-CCvsCC_3vs3Males_P60_CCvsCC_Group9V_30min_interaction_cam_2_Fri_06_07_2019_mjpeg_tracks.mat
../data/mat_files/dbs_Head-Tail_20191013.mat
----------------

# power centrality z-score

In [42]:
din = [0,1,2,3,4,5]
dout = [5,2,4,3,1,0]
results=np.zeros((6,1000))
for i in range(100):
    done=False
    while not done:
        try:
            #results[:,i] gets column i
            results[:,i]=np.array(list(bonacich_power_centrality(nx.directed_configuration_model(din,dout, create_using=nx.DiGraph()), 1, 1e-07, False).values()))
            done=True
        except:
            False

print(np.mean((results[:,1]-np.mean(results[:,1:],1))/np.std(results[:,1:])))

[ 0.1699974  -4.37496059  2.61839886 -5.71057608  0.92970654  0.        ]


In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 0, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, -1, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 1, 1e-07,True, False, 2500)

In [ ]:
%%time
# def all_treatment_analysis(treatments, All, network_measures, reverse, exponent, tol, rescale, raw, iterations):
# treatments = [["./data/dbs_Head-Tail_20191013.mat"], ["./data/dbs_Head-Head_20191013.mat"], ["./data/dbs_Centers-Close_20191013.mat"], ["./data/dbs_Oriented-Towards_20191013.mat"]]
treatments = [["./data/dbs_Head-Tail_20191013.mat"]]
for x in treatments:
    print(x)
    df_zscore_ev_split = all_treatment_analysis(x, True, [['bonacich power centrality', "NA"]], False, 0.1, 1e-07,True, False, 2500)